# Installation

## I don't know what you've installed or how you've installed it, so let's talk before you run any of this.

**OS X folks** can run the following:

* `brew install geos`
* `brew install gdal`
* `brew install spatialindex`
* `pip3 install pillow`
* `pip3 install pysal`
* `pip3 install geopandas`
* `pip3 install https://github.com/matplotlib/basemap/archive/v1.0.7rel.tar.gz`
* `pip3 install rtree`

For **Windows without Anaconda**, [use this guide](http://geoffboeing.com/2014/09/using-geopandas-windows/) to install through `pip` directly from `whl` files.

# Geopandas Usage

## Importing

You'll be importing

* pandas because you love it
* geopandas for geographic stuff
* `Point` from shapely to help convert CSV files into something geopandas can understand

and `%matplotlib inline` for viewing maps, of course.

In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

%matplotlib inline

## Opening a shapefile

Let's open up the Community Districts data. **What kind of file is it?**

In [4]:
districts = gpd.read_file("Community Districts/districts.shp")

OSError: no such file or directory: 'Community Districts/districts.shp'

## Introduction to the GeoDataFrame

A GeoDataFrame is *just like a dataframe*, it just... has geographic stuff in it.

In [ ]:
districts.head()

In [ ]:
districts[districts.boro_cd > 400]

In [ ]:
districts.head()

## Visualizing a shapefile

You can just use `.plot()` to visualize a GeoDataFrame, it's nice and easy.

In [ ]:
districts.plot()

## Changing the CRS (Coordinate Reference System)

### WAY ONE: Just changing the Projection

In [ ]:
# Go into the crs to convert it...
# ignore the datum and spheroid,
# just change the PROJECTION to MERCATOR
districts.to_crs({'proj': 'merc'}).plot()

In [ ]:
# Give it the SECRET CODE from the PETROLEUM GROUP
# (which you can try to find by googling)
# (or hopefully you have a list because they're
# all very confusingly/similarly named)
districts.to_crs(epsg=3857).plot()

## Opening a CSV of points

geopandas doesn't understand a CSV file of lat/lon points, so you need to convert each line into shapely geometry, then feed that into a new geo dataframe.

Once you do that, you need to set the `crs` to `{'init': 'epsg:4326'}` so it knows what kind of datum/sphereoid/projection you're measuring from.

**Let's try opening the earthquakes CSV**

In [ ]:
df = pd.read_csv("earthquakes_1.0_day.csv")
df.head(2)

In [ ]:
def make_point(row):
    return Point(row.longitude, row.latitude)

# Go through every row, and make a point out of its lat and lon
points = df.apply(make_point, axis=1)

# Make a new GeoDataFrame
# using the data from our old df
# but also adding in the geometry we just made
earthquakes = gpd.GeoDataFrame(df, geometry=points)

# It doesn't come with a CRS because it's a CSV, so let's
# say "hey, let's use the standard shape of the earth etc"
earthquakes.crs = {'init': 'epsg:4326'}

# Let's look at the first few
earthquakes.head()

In [ ]:
earthquakes.plot()

In [ ]:
# Read in the CSV
df = pd.read_csv("earthquakes_1.0_day.csv")

points = df.apply(lambda row: Point(row.longitude, row.latitude), axis=1)
earthquakes = gpd.GeoDataFrame(df, geometry=points)
earthquakes.crs = {'init': 'epsg:4326'}

# If you want to know how this all works, look above
earthquakes.head(2)

In [ ]:
earthquakes.plot(figsize=(20,5))

## Using the built-in map

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
ax = world.plot(figsize=(20,5))
ax.axis('off')

In [ ]:
world.crs

# Styling your visuals

## Setting size, line and shape colors, widths, axes

* `linewidth`
* `color`
* `edgecolor`
* `ax.axis`

Let's plot the community districts!

In [ ]:
ax = world.plot(figsize=(20,5), linewidth=0.25, edgecolor='white', color='lightgrey')
ax.axis('off')

Let's plot the world!

In [ ]:
ax = districts.to_crs(epsg=3857).plot(figsize=(20,5), linewidth=0.25, edgecolor='white', color='pink')
ax.axis('off')

## Setting the projection

You can use `to_crs` to convert to different projections. In typical pandas fashion, you can do it a lot of ways, but the easiest is to send a `epsg=` and feed it the correct EPSG code.

You'll also probably want to do an `ax.axis('off')` to turn off the splines and axes!

What are the EPSG codes for some common projections?

In [ ]:
earthquakes.plot(figsize=(20,7))

## Styling markers

* markersize
* color
* alpha

In [ ]:
# Plot the earthquakes:
# BIGGER CIRCLES
# all one color
# make them a little transparent
# NO AXES OR BORDERS AROUND THE MAP!!!!!!!

ax = earthquakes.plot(figsize=(20,5), markersize=10, color='pink', alpha=0.5)
ax.axis('off')

In [ ]:
# Plot the earthquakes:
# BIGGER CIRCLES
# all one color
# make them a little transparent
# NO AXES OR BORDERS AROUND THE MAP!!!!!!!

# Save the first layer as ax
ax = world.plot(color='lightgrey', linewidth=0.5, edgecolor='white', figsize=(15,5))
# Pass ax=ax to the second layer
earthquakes.plot(markersize=10, color='pink', alpha=0.5, ax=ax)
ax.axis('off')

# Colormaps/ramps

## Auto colormap

Giving your `plot` a `column` and a `cmap` will colorize your values. You can try `plasma` as your color map, or check out [more here](https://matplotlib.org/examples/color/colormaps_reference.html).

In [ ]:
earthquakes.head(2)

In [ ]:
# Plot the earthquakes:
# BIGGER CIRCLES
# all one color
# make them a little transparent
# NO AXES OR BORDERS AROUND THE MAP!!!!!!!

# Save the first layer as ax
ax = world.plot(color='lightgrey', linewidth=0.5, edgecolor='white', figsize=(15,5))
# Pass ax=ax to the second layer
earthquakes.plot(markersize=10, alpha=0.5, ax=ax, column='mag')
ax.axis('off')

In [ ]:
# Plot the earthquakes:
# BIGGER CIRCLES
# all one color
# make them a little transparent
# NO AXES OR BORDERS AROUND THE MAP!!!!!!!

# Save the first layer as ax
ax = world.plot(color='lightgrey', linewidth=0.5, edgecolor='white', figsize=(15,5))
# Pass ax=ax to the second layer
earthquakes.plot(markersize=10, alpha=0.5, ax=ax, column='mag', cmap='plasma')
ax.axis('off')

In [ ]:
world.head(2)

## Auto colormap again

We can also try with the world. What's the `gdp_md_est` column looking like?

In [ ]:
world.plot(column='gdp_md_est', cmap='inferno')

In [ ]:
world.plot(column='pop_est', cmap='inferno')

## Plotting multiple layers of data

Let's try plotting the earthquakes on top of the world. **Save your first plot as `ax` and send it to the next one as `ax=ax`.**

# Setting the projection by `proj` with named projections

Instead of using an EPSG code, you can also set the projection with `to_crs` by  `.to_crs({'proj': 'merc'})` or something similar.

I don't recommend this method, but it is a little friendlier than EPSG codes.

### Plot the world with the default projection

In [ ]:
ax = world.plot()
ax.set_title("Default")

### Plot the world with Mercator (merc)

In [ ]:
ax = world.to_crs({'proj': 'merc'}).plot()
ax.set_title("Default")

### Plot the world with [Transverse Mercator](https://en.wikipedia.org/wiki/Transverse_Mercator_projection) (tmerc)

In [ ]:
ax = world.to_crs({'proj': 'tmerc'}).plot()
ax.set_title("Default")

### Plot the world with Albers Equal Area (aea)

In [ ]:
ax = world.to_crs({'proj': 'aea'}).plot()
ax.set_title("Default")

# Spatial join

# Dataset 1: States

Let's import the states and clean them up a little bit. we need to clean the data up a little

In [ ]:
# Read in the shapefile from cb_2016_us_state_500k as "states"
states = gpd.read_file("cb_2016_us_state_500k/cb_2016_us_state_500k.shp")

In [ ]:
# Get rid of Guam, Mariana Islands and Virgin Islands
states = states[states.STATEFP.astype(int) < 60]
# Get rid of Hawaii and Alaska
states = states[~states.NAME.isin(['Hawaii', 'Alaska'])]
states.tail(5)

## Dataset 2: Waffle House

Read in `wafflehouses.csv`, and convert it to a GeoDataFrame.

In [ ]:
# Read in the CSV
df = pd.read_csv("wafflehouses.csv")

points = df.apply(lambda row: Point(row.long, row.lat), axis=1)
wafflehouses = gpd.GeoDataFrame(df, geometry=points)
wafflehouses.crs = {'init': 'epsg:4326'}

# If you want to know how this all works, look above
wafflehouses.head(2)

### Plot the locations, coloring based on the 'score' column.

In [ ]:
ax = states.plot()
wafflehouses.plot(column='score', ax=ax)

# The actual spatial join

### Is the CRS of the states the same as the CRS of the Waffle House locations?

In [ ]:
states.crs

In [ ]:
wafflehouses.crs

### If not, we'll force them to match using `.to_crs`

In [ ]:
# Convert CRS to match
converted_states = states.to_crs(wafflehouses.crs)
converted_states.head(2)

In [ ]:
converted_states.crs

In [ ]:
wafflehouses.crs

### And now we can join

Open up Terminal and run

* pip install rtree
* brew install spatialindex

In [ ]:
wafflehouses.head()

In [ ]:
converted_states.head(2)

In [ ]:
converted_states.plot()

In [ ]:
wafflehouses.plot()

In [ ]:
# http://geopandas.org/mergingdata.html
wafflehouses_with_state_data = gpd.sjoin(wafflehouses, converted_states, how='left', op='within')
wafflehouses_with_state_data.head()

In [ ]:
wafflehouses_with_state_data['NAME'].value_counts()

## Doing things with spatially joined data

* What column do we use for color?
* Add a legend with `legend=True`
* Something is going to go wrong, though!

In [ ]:
# Need dropna because some wafflehouses are missing states
wafflehouses_with_state_data.dropna().plot(column='NAME', markersize=10, figsize=(20,5))

## What if we reverse the spatial join and make it 'contains'?

How is this different than what we did before?

In [ ]:
# http://geopandas.org/mergingdata.html
states_with_wafflehouse_data = gpd.sjoin(converted_states, wafflehouses, how='left', op='contains')
states_with_wafflehouse_data.head()

In [ ]:
states_with_wafflehouse_data.shape

In [ ]:
select_columns = states_with_wafflehouse_data[['NAME', 'geometry', 'score']]
select_columns.head()

## Aggregating with `.dissolve` (the geographic version of 'groupby')

http://geopandas.org/aggregation_with_dissolve.html

### In theory we'd run the following line

But it doesn't work because we have too much data, and `.dissolve` isn't smart enough to deal with it.

In [ ]:
# We can't do this, I think because there are too many wafflehouses
#wafflehouse_counts = states_with_wafflehouse_data.dissolve(by='NAME', aggfunc='count')
#wafflehouse_counts.head()

### But we can try it out with the first 5/20/50 of them

In [ ]:
%%time
select_columns.head(5).dissolve(by='NAME', aggfunc='count')

In [ ]:
%%time
select_columns.head(20).dissolve(by='NAME', aggfunc='count')

In [ ]:
%%time
select_columns.head(50).dissolve(by='NAME', aggfunc='count')

In [ ]:
%%time
select_columns.head(200).dissolve(by='NAME', aggfunc='count')

## Spatial joins for LARGE data sets (NOT using dissolve)

Instead of using `.dissolve`, we need to use `.contains` to say "find me all of the waffle houses inside of this one specific state". I don't know why this works better, but it does. We'll use `.sum()` to count the number inside, but you could also do something like `['score'].mean()` etc.

### First, let's try it with one state

In [ ]:
states.head(2)

In [ ]:
states.loc[0].geometry

In [ ]:
# False = 0
# True = 1
# Count the number of states where the name contains "New"
states.NAME.str.contains("New").sum()

In [ ]:
# Give me the first state!
state = states.loc[0]
# Look at the wafflehouses...
# are they inside of the state's geometry?
wafflehouses.within(state.geometry).sum()

In [ ]:
# So for our first state, there were 147 inside of there

### Now, let's try it with every state

In [ ]:
# You can use .contains
# counts the true ones
states['wafflehouse_count'] = states.apply(lambda state: wafflehouses.within(state.geometry).sum(), axis=1)
states.head()

In [ ]:
states.plot(column='wafflehouse_count', scheme='quantiles')

In [ ]:
states.plot(column='wafflehouse_count', scheme='equal_interval')

In [ ]:
states.plot(column='wafflehouse_count', scheme='fisher_jenks', legend=True)

## Spatial joins for SMALLER data sets (YES using dissolve)

If our dataset isn't that big, though, we're fine to use `.dissolve`.

In [ ]:
# Each one is a country
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head(10)

See how every country has a continent? We can `.dissolve` to group them together based on continent. It's like groupby, really!

In [ ]:
# But we'll dissolve them so it's only continents
continents = world.dissolve(by='continent', aggfunc='sum')
continents.head()
continents.plot(column='pop_est')

# Saving the results

You want to look at this stuff in Leaflet, right? For that we'll need to save. Geopandas supports practically _every_ file format you could ever want.

In [ ]:
#wafflehouses_with_state_data.to_file("wafflehouses.json", driver='GeoJSON')